In [1]:
#Dependencies
import pandas as pd

In [2]:
#Set the file path
colorado_county_population_path="../../Data_files/raw_data/colorado_raw_data/colorado_population_bycounty.csv"
colorado_county_crime_path="../../Data_files/raw_data/colorado_raw_data/colorado_county_crimecounts.csv"
colorado_county_vote_path = "../../Data_files/raw_data/colorado_raw_data/county_vote_CO.csv" 
colorado_county_unemployment_path = "../../Data_files/raw_data/colorado_raw_data/colorado_unemployment.csv"

In [3]:
# Read the csv file into dataframe
colorado_county_raw_population_df=pd.read_csv(colorado_county_population_path,header=3)
colorado_county_raw_crime_df=pd.read_csv(colorado_county_crime_path)
colorado_county_raw_vote_df=pd.read_csv(colorado_county_vote_path)
colorado_county_raw_unemployment_df= pd.read_csv(colorado_county_unemployment_path,header=5)

## Cleanup Process of Colorado Population Data

In [4]:
# Show the last 10 rows
colorado_county_raw_population_df.tail(10)

,Unnamed: 0,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
60,".Summit County, Colorado","27,994","27,994","28,073","28,042","28,298","28,728","29,317","30,045","30,552","30,820","30,817","31,011"
61,".Teller County, Colorado","23,350","23,359","23,469","23,369","23,445","23,356","23,433","23,388","24,058","24,674","25,113","25,388"
62,".Washington County, Colorado","4,814","4,812","4,821","4,762","4,701","4,743","4,733","4,786","4,834","4,921","4,888","4,908"
63,".Weld County, Colorado","252,825","252,827","254,207","258,827","264,131","270,165","277,112","285,819","295,712","305,885","314,815","324,492"
64,".Yuma County, Colorado","10,043","10,043","10,049","10,134","10,100","10,132","10,184","10,037","10,055","9,959","9,947","10,019"
65,Note: The estimates are based on the 2010 Cens...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,Suggested Citation:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,Annual Estimates of the Resident Population fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,"Source: U.S. Census Bureau, Population Division",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,Release Date: March 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Select necessary columns
colorado_county_raw_population_df=colorado_county_raw_population_df[["Unnamed: 0","2018"]]
# Drop unnecessary rows
colorado_county_raw_population_df=colorado_county_raw_population_df.drop(0)
colorado_county_raw_population_df=colorado_county_raw_population_df.drop([65,66,67,68,69])
# Rename columns
colorado_county_population_df=colorado_county_raw_population_df.rename(columns={"Unnamed: 0":"County","2018":"Population"})

In [6]:
colorado_county_population_df.head()

,County,Population
1,".Adams County, Colorado","511,354"
2,".Alamosa County, Colorado","16,248"
3,".Arapahoe County, Colorado","651,797"
4,".Archuleta County, Colorado","13,730"
5,".Baca County, Colorado","3,584"


## Cleanup Process of Colorado Crime Data

In [7]:
# Loop through each row in dataframe, leave only the county names
for index,row in colorado_county_population_df.iterrows():
    county_state=row["County"].replace(" County, Colorado","")
    county=county_state.replace(".","")
    row["County"]=county

In [8]:
colorado_county_raw_crime_df.head()

,Metropolitan/Nonmetropolitan,County,Violent\ncrime,Property\ncrime
0,Metropolitan Counties,Arapahoe,350.0,"1,865"
1,NaN,Boulder,78.0,672
2,NaN,Clear Creek,24.0,56
3,NaN,Douglas,366.0,"2,489"
4,NaN,Elbert,19.0,3


In [9]:
# Select necessary columns
colorado_county_crime_df=colorado_county_raw_crime_df[["County","Violent\ncrime","Property\ncrime"]]
colorado_county_crime_df.head()

,County,Violent\ncrime,Property\ncrime
0,Arapahoe,350.0,"1,865"
1,Boulder,78.0,672
2,Clear Creek,24.0,56
3,Douglas,366.0,"2,489"
4,Elbert,19.0,3


In [10]:
# Drop row no.46
colorado_county_crime_df=colorado_county_crime_df.drop(46)

In [11]:
# Loop through the rows and remove space at the end of the county name
for index,row in colorado_county_crime_df.iterrows():
    county=colorado_county_crime_df.loc[index,"County"]
    colorado_county_crime_df.loc[index,"County"]=county.strip()

In [12]:
# Rename the columns
colorado_county_crime_df=colorado_county_crime_df.rename(columns={"Violent\ncrime":"Violent Crime","Property\ncrime":"Property Crime"})

## Process of Merging Colorado Crime and Population Data

In [13]:
# Merge dataframes
colorado_county_combined_df=pd.merge(colorado_county_crime_df,colorado_county_population_df,on="County")
colorado_county_combined_df.head()

,County,Violent Crime,Property Crime,Population
0,Arapahoe,350.0,"1,865","651,797"
1,Boulder,78.0,672,"324,636"
2,Clear Creek,24.0,56,"9,604"
3,Douglas,366.0,"2,489","342,989"
4,Elbert,19.0,3,"26,303"


In [14]:
# Check the datatypes
colorado_county_combined_df.dtypes

County             object
Violent Crime     float64
Property Crime     object
Population         object
dtype: object

In [15]:
# Remove "," from the numbers
colorado_county_combined_df["Population"]=colorado_county_combined_df["Population"].str.replace(",","")
colorado_county_combined_df["Property Crime"]=colorado_county_combined_df["Property Crime"].str.replace(",","")
# Change the datatype to numeric
colorado_county_combined_df["Population"]=pd.to_numeric(colorado_county_combined_df["Population"])
colorado_county_combined_df["Property Crime"]=pd.to_numeric(colorado_county_combined_df["Property Crime"])

In [16]:
# Convert the total crime numbers to crime rate per 1000
vc_rate = round(colorado_county_combined_df["Violent Crime"]/(colorado_county_combined_df["Population"]/1000),3)
pc_rate = round(colorado_county_combined_df["Property Crime"]/(colorado_county_combined_df["Population"]/1000),3)

In [17]:
# Create new columns and assign values
colorado_county_combined_df["Violent Crime per 1,000 inhabitants"]=vc_rate
colorado_county_combined_df["Property Crime per 1,000 inhabitants"]=pc_rate

In [18]:
# Drop total crime column
colorado_county_combined_df=colorado_county_combined_df.drop(["Violent Crime","Property Crime"], axis=1)
colorado_county_combined_df.head()

,County,Population,"Violent Crime per 1,000 inhabitants","Property Crime per 1,000 inhabitants"
0,Arapahoe,651797,0.537,2.861
1,Boulder,324636,0.240,2.070
2,Clear Creek,9604,2.499,5.831
3,Douglas,342989,1.067,7.257
4,Elbert,26303,0.722,0.114


## Cleanup Process of Colorado Voting Data

In [19]:
colorado_county_raw_vote_df.head()

,county,dem_gov_2014_%,rep_gov_2014_%,dem_house_2012_%,rep_house_2012_%,dem_house_2014_%,rep_house_2014_%,dem_house_2016_%,rep_house_2016_%,dem_pres_2012_%,rep_pres_2012_%,dem_pres_2016_%,rep_pres_2016_%,dem_senate_2014_%,rep_senate_2014_%,dem_senate_2016_%,rep_senate_2016_%,label_3
0,Adams,0.501126,0.444168,0.526886,0.401861,0.511630,0.470264,0.514574,0.426280,0.571123,0.402724,0.498552,0.413482,0.476951,0.448761,0.526509,0.412360,DEM
1,Alamosa,0.509386,0.433133,0.486207,0.458622,0.425093,0.515413,0.495194,0.462774,0.567535,0.402829,0.459576,0.438968,0.474155,0.456665,0.529977,0.415089,DEM
2,Arapahoe,0.520078,0.439368,0.485311,0.450000,0.455291,0.490463,0.457761,0.478489,0.539042,0.439863,0.527590,0.386252,0.480201,0.464848,0.540040,0.409837,DEM
3,Archuleta,0.394250,0.555210,0.339208,0.595602,0.297932,0.634433,0.356941,0.597823,0.397832,0.574993,0.340646,0.581006,0.372272,0.569045,0.379206,0.561500,Swing
4,Baca,0.217578,0.725616,0.247859,0.698237,0.178631,0.754591,0.142857,0.815436,0.221747,0.740266,0.131444,0.814213,0.189887,0.739107,0.269878,0.687091,REP


In [20]:
colorado_county_raw_vote_df=colorado_county_raw_vote_df[["county","label_3"]]

In [21]:
colorado_county_vote_df = colorado_county_raw_vote_df.rename(columns={"county": "County", "label_3": "Party"})

colorado_county_vote_df["Affiliated Party Color"]= ""

# set condition to assign each row for new column
colorado_county_vote_df.loc[colorado_county_vote_df["Party"] == "REP", "Affiliated Party Color"] = "Red"
# separate by color for merge- purple
colorado_county_vote_df.loc[colorado_county_vote_df["Party"] == "Swing", "Affiliated Party Color"] = "Purple"
# blue state
colorado_county_vote_df.loc[colorado_county_vote_df["Party"] == "DEM", "Affiliated Party Color"] = "Blue"
colorado_county_vote_df.head()

,County,Party,Affiliated Party Color
0,Adams,DEM,Blue
1,Alamosa,DEM,Blue
2,Arapahoe,DEM,Blue
3,Archuleta,Swing,Purple
4,Baca,REP,Red


In [22]:
colorado_county_color_df = colorado_county_vote_df[["County", "Affiliated Party Color"]]

colorado_county_color_df.head()

,County,Affiliated Party Color
0,Adams,Blue
1,Alamosa,Blue
2,Arapahoe,Blue
3,Archuleta,Purple
4,Baca,Red


## Cleanup Process of Colorado Unemployment Data

In [23]:
# Display first 5 rows
colorado_county_raw_unemployment_df.tail()

,Rank,Area,Unemployment Rate
61,62,Baca County,1.8%
62,63,Cheyenne County,1.8%
63,64,Kiowa County,1.8%
64,"Source: LAUS Unit, LAUS system output file",NaN,NaN
65,Downloaded: 05/01/2020 2:05 PM,NaN,NaN


In [24]:
# Remove unnecessary columns and rows
colorado_county_unemployment_df=colorado_county_raw_unemployment_df.drop([64,65]).drop("Rank",axis=1)
colorado_county_unemployment_df

,Area,Unemployment Rate
0,Huerfano County,6.4%
1,Fremont County,5.0%
2,Otero County,4.9%
3,Rio Grande County,4.8%
4,Pueblo County,4.7%
...,...,...
59,Phillips County,1.9%
60,Yuma County,1.9%
61,Baca County,1.8%
62,Cheyenne County,1.8%


In [25]:
# Remove "%" for later data analysis
colorado_county_unemployment_df["Unemployment Rate"]=colorado_county_unemployment_df["Unemployment Rate"].str.replace("%","")
# Remove " County"
colorado_county_unemployment_df["Area"]=colorado_county_unemployment_df["Area"].str.replace(" County","")
# Rename the column
colorado_county_unemployment_df=colorado_county_unemployment_df.rename(columns={"Area":"County"})
# Sort the county names and reset index
colorado_county_unemployment_df=colorado_county_unemployment_df.sort_values(by="County").reset_index(drop=True)
# Change the datatype to numeric
colorado_county_unemployment_df["Unemployment Rate"]=pd.to_numeric(colorado_county_unemployment_df["Unemployment Rate"])
colorado_county_unemployment_df

,County,Unemployment Rate
0,Adams,3.3
1,Alamosa,3.9
2,Arapahoe,3.1
3,Archuleta,3.3
4,Baca,1.8
...,...,...
59,Summit,2.1
60,Teller,3.6
61,Washington,2.4
62,Weld,2.9


## Process of Merging Remaining Data

In [26]:
# Merge vote dataframe and unemployment dataframe 
vote_unemployment_merge = pd.merge(colorado_county_color_df, colorado_county_unemployment_df, on= "County")
vote_unemployment_merge.tail()

,County,Affiliated Party Color,Unemployment Rate
59,Summit,Blue,2.1
60,Teller,Red,3.6
61,Washington,Red,2.4
62,Weld,Purple,2.9
63,Yuma,Red,1.9


In [27]:
# Merge the vote&unemployment dataframe with the crime&population dataframe
colorado_county_merge = pd.merge(vote_unemployment_merge, colorado_county_combined_df, on="County")
colorado_county_merge.head()

,County,Affiliated Party Color,Unemployment Rate,Population,"Violent Crime per 1,000 inhabitants","Property Crime per 1,000 inhabitants"
0,Arapahoe,Blue,3.1,651797,0.537,2.861
1,Archuleta,Purple,3.3,13730,3.569,6.846
2,Baca,Red,1.8,3584,0.837,0.558
3,Bent,Purple,3.1,5808,5.510,32.197
4,Boulder,Blue,2.7,324636,0.240,2.070


In [28]:
colorado_county_merge.to_csv("../../Data_files/clean_data/colorado_clean_data/colorado_county_data(combined_info).csv", index=False)
colorado_county_merge

,County,Affiliated Party Color,Unemployment Rate,Population,"Violent Crime per 1,000 inhabitants","Property Crime per 1,000 inhabitants"
0,Arapahoe,Blue,3.1,651797,0.537,2.861
1,Archuleta,Purple,3.3,13730,3.569,6.846
2,Baca,Red,1.8,3584,0.837,0.558
3,Bent,Purple,3.1,5808,5.510,32.197
4,Boulder,Blue,2.7,324636,0.240,2.070
5,Chaffee,Purple,2.6,20041,0.549,2.994
6,Cheyenne,Red,1.8,1859,0.000,0.538
7,Clear Creek,Blue,2.9,9604,2.499,5.831
8,Conejos,Blue,4.1,8181,0.611,3.789
9,Crowley,Red,4.2,5957,1.847,7.051
